# Evaluating Model Fitness

## Detecting Overfitting and Underfitting

## Objectives
* Compare training and testing scores to detect overfit/underfit models
* Use best practices to reduce overfitting in models
* Use best practices to reduce underfitting in models

## Evaluating Model Fitness

In this lab, we're going to build a classifier on a small subset of the titanic dataset.  We'll make use of training and testing validation methods that we learned in previous labs to detect when our model is overfit or underfit.  We'll then implement strategies to fix our overfit and underfit models.  

We'll start by importing all the libraries we'll need, and reading in the Titanic Dataset.  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
# Read in dataset and store in a dataframe
raw_df = pd.read_csv('titanic.csv')
raw_df.head()

Next, we'll do some brief cleaning to make the dataset more amenable to classification algorithms.  

We'll remove the `Ticket` and `Cabin` columns, and one-hot encode the `Embarked` columns.  We'll also convert convert the `Sex` column to binary encoding, replace all `'male'` instances with `0`, and `'female'` instances with 1.

Run the cell below to clean the dataset. 

In [ ]:
# Data Cleaning Step 1: Drop Unnecessary columns (Ticket and Cabin)
df_step1 = raw_df.drop(['Ticket', 'Cabin', 'PassengerId'], axis=1)

# Data Cleaning Step 2: Binary encode Sex column (male = 0, female = 1)
df_step1['Sex'] = df_step1.Sex.map({'male': 0, 'female': 1})

# Data Cleaning Step 3: One-hot encode categorical column (embarked) into separate columns for each possible value
clean_df = pd.get_dummies(df_step1)

# Data Cleaning Step 4: Remove rows with null values
clean_df.dropna(inplace=True)

# Data Cleaning Step 5: Store target column in a separate variable and then remove from dataframe
target = clean_df['Survived']
clean_df.drop('Survived', axis=1, inplace=True)

# Data Cleaning Step 6: Inspect everything to make sure that we have no null values, and everything looks correct
print(clean_df.describe())
clean_df.head()

## Model Fitness

When we say a model is overfit or underfit, we're talking about how well it generalizes to what's in the data.  it's easiest to understand by looking at the following visualization:

<center><img src='overfit-underfit-graphic.png'></center>

As we can see in the graphic above:

The model on the left **_underfits_** the data.  This model will make mistakes because it is too simple to capture the complexity in the data, since the model is linear but the data is not.  

The model on the right **_overfits_** the data.  This model will make mistakes because it has mistakenly interpreted random differences in the training data as meaningful, causing it to add many extra twists and turns.  When this model is used on real world data, it will often be wrong because of these loops, since the data points in the training data that these extra twists and turns go out of their way to fit are not special, and will not occur in the overall population the way that it did in the sample that makes up the training set.  This model will lose accuracy on real-world data at the expense of scoring very highly on the training set.  

The model in the middle has a **_good fit_**.  We can see that the line generally fits the underlying pattern in the data.  Although some data points don't fall on the line due to natural "noise" (variance) in the data set, these points are equally distributed above and below the line, meaning that on average, this model is as accurate as it can be.  

## Underfit Models

The most obvious dead-giveaway that our model has underfit the data is when it does poorly on both the training and the testing sets.  This usually suggests that our model does not adequately capture the underlying complexity in the data--put simply, the model is too simple.  

The easiest way to deal with underfitting is to create a more complex model.  Refer to the examples in the image above--the underfit model assumes a simple, linear fit.  The correct model has a 2nd degree polynomial fit to capture the nonlinearity in the model.    
    
In practice, an underfit model often means a model that doesn't have enough features to adequately make predictions about the data.  We'll build a small underfit **_Decision Tree Classifier_** on the dataset below, but we'll only use only a few features.  

We'll then increase the complexity of the model by adding in a few other dimensions to help the model make predictions.  This will add enough complexity to our model to see our training and testing scores goes up, suggesting that the model in no longer underfit.  

In [ ]:
underfit_df = clean_df[['Pclass']]
underfit_clf = DecisionTreeClassifier()
underfit_results_dict = cross_validate(underfit_clf, underfit_df, target, cv=10, return_train_score = True)

print("Average Training Score: {}".format(np.mean(underfit_results_dict['train_score'])))
print("Average Testing Score: {}".format(np.mean(underfit_results_dict['test_score'])))

plt.figure()
axes = plt.gca()
x = [i for i in range(1, 11)]
plt.bar(x, underfit_results_dict['train_score'])
plt.bar(x, underfit_results_dict['test_score'])
axes.set_ylim([0.25, 1.0])
plt.legend(['Training', 'Testing'])
plt.xlabel("Models Built During Cross Validation")
plt.ylabel("Accuracy Score")
plt.title("Individual Model Performance (Underfit Models)")

Note that each integer on the X-axis of the graph above corresponds to the score of the model built during that cross validation step. Each model is independent of one another, and the order doesn't matter.

That's pretty low, suggesting that the model our Decision Tree Classifier has learned doesn't fit the data as well as it could. 

Let's add in some other columns and fit a new model, and compare the performance of the two. 

In [ ]:
overfit_clf = DecisionTreeClassifier()
overfit_results_dict = cross_validate(overfit_clf, clean_df, target, cv=10, return_train_score=True)

print("Average Training Score: {}".format(np.mean(overfit_results_dict['train_score'])))
print("Average Testing Score: {}".format(np.mean(overfit_results_dict['test_score'])))

plt.figure()
axes = plt.gca()
plt.bar(x, overfit_results_dict['train_score'])
plt.bar(x, overfit_results_dict['test_score'])
axes.set_ylim([0.25, 1.0])
plt.legend(['Training', 'Testing'])
plt.xlabel("Models Built During Cross Validation")
plt.ylabel("Accuracy Score")
plt.title("Individual Model Performance (Overfit Models)")

Our model performance has improved by ~11% on our testing set, and 31% on our training set.  Our model is no longer underfit, but now we have a new problem--overfitting!

## Overfit Models

In our results from our improved model, we can see a tell-tale sign of overfitting: The training score is significantly higher than the testing score.  

Let's think about why this tells us that the model has overfit the training data.  The **_testing score_** tells us how well the model can accurately predict new examples it hasn't seen before.  If we gave the model new data other passengers on the Titanic, we could expect the model to predict their survival with ~77% accuracy.

But on our training set, the model manages to predict survival with over 98% accuracy.  

So why does the model score 21% better on examples from the training set than the testing set? Where does this extra "accuracy" come from?

The answer is that the model has overfit the data.  Recall that our model learned to make predictions by examining the data in the training set, looking for patterns that result in successful predictions.  There is guaranteed to be some randomness in the training data that the model will accidentally mistake for important.  For instance, in the training set, the model may realize that 100% of male passengers in Pclass 3 that embarked from Queensland (Embarked_Q) survived.  We can immediately tell that this is a coincidence, and that this doesn't actually help us with predictions.  Although the model may have noticed that all 5 or 6 passengers that fit this pattern in the training data survive, we can wisely realize that this is randomness and coincidence that these passengers all happened in the sample that we used as our training data--there are likely many more poor male passengers from Queensland that died, they just aren't in our training set.  

The model has mistakenly "learned" these spurious patterns, because during training, these rules seem just as helpful as the rules that actually do provide predictive accuracy--rules such as "women/children/rich people are more likely to survive".  It is important to note that mathematically, the model has no real way of telling the two apart.  

We'll fix this overfit model by adding in some constraints that force our model to keep it simple, and avoid learning incorrect patterns that only apply to 3 or 4 instances in the training data.  

For our Decision Tree Classifier, we'll do this by pre-pruning our tree, telling it that branches (decisions) are only allowed to grow if they have enough examples to suggest this is likely a real pattern, and not a coincidence in the training data. 

Run the cell below to create this new model that avoids overfitting.  

In [ ]:
good_fit_clf = DecisionTreeClassifier(min_samples_split=15, min_samples_leaf=25, max_depth=7)
good_fit_results_dict = cross_validate(good_fit_clf, clean_df, target, cv=10, return_train_score=True)

print("Average Training Score: {}".format(np.mean(good_fit_results_dict['train_score'])))
print("Average Testing Score: {}".format(np.mean(good_fit_results_dict['test_score'])))

plt.figure()
axes = plt.gca()
plt.bar(x, good_fit_results_dict['train_score'])
plt.bar(x, good_fit_results_dict['test_score'])
axes.set_ylim([0.25, 1.0])
plt.legend(['Training', 'Testing'])
plt.xlabel("Models Built During Cross Validation")
plt.ylabel("Accuracy Score")
plt.title("Individual Model Performance (Good Fit)")

These models are much better--we've improved our testing accuracy by a few percent over the overfit models, but we've also brought the training accuracy down to the same general level as our testing accuracy, suggesting that our model will not make mistaken classification on real world data due to some mistaken patterns it thought it found in the training set. 

### Other Ways to Overfit

A quick caveat on overfitting--there are other things that can cause a model to overfit, and other strategies for dealing with overfitting that are outside the scope of this lab.  The example above is not an exhaustive exploration of the topic.  

Other ways to deal with overfitting include:

* Removing features from the data set (reducing it _dimensionality_), only including the most important ones
* Adding more training data (when possible)
* Using _regularization_ to penalize model complexity

Dealing with overfitting in models is typically much harder than dealing with underfitting, and there are many different causes and solutions worth exploring on this topic.  If you'd like to explore this topic further, you'll find [this quora answer](https://www.quora.com/How-can-I-avoid-overfitting) from Machine Learning expert Sebastian Raschka is a great place to start. 

# Summary

In this lab, we learned about:

* How to detect **_Overfitting_** and **_Underfitting_** in our models by comparing training and testing scores.
* Strategies for improving model fitness and avoiding underfit/overfit models.  